# GENERATING MAIN OUTPUT TABLES

This notebook contains the code used to generate the main output tables from the 1,000s of files that FRAGSYS generates. Due to a size constraint, these files are not in the repository. However, the main tables generated here are, and are the basis of the analysis.

## IMPORTING NECESSARY PACKAGES

In [1]:
from fragsys_analysis import *

## READING INPUT DATA

In [10]:
main_dir = "/cluster/gjb_lab/2394007/pandda_analysis/phase4/prots"
prot_dirs = sorted(Path(main_dir).iterdir(), key = os.path.getmtime)
results_dir = "./../results/"

## PROTEIN SUMMARY DATAFRAME

In [11]:
results_dfs = []
for prot_dir in prot_dirs:
    prot = str(prot_dir).split("/")[-1]
    subdirs = sorted(os.listdir(os.path.join(prot_dir, "unsupp_cifs"))) # one for each group
    for subdir in subdirs:
        res_path = os.path.join(prot_dir, "{}_{}_results.csv".format(prot, subdir))
        if os.path.isfile(res_path):
            prot_res = pd.read_csv(res_path)
            results_dfs.append(prot_res)
        else:
            print("Group {} of {} did not present results".format(subdir, prot))
            pass
        
fragsys_results_df = pd.concat(results_dfs).reset_index(drop = True)
fragsys_results_df["vars_per_seq"] = round(fragsys_results_df.n_vars/fragsys_results_df.n_human_seqs, 2)
fragsys_results_df["vars_per_res"] = round(fragsys_results_df.n_vars/fragsys_results_df.n_human_res, 2)

Group 1 of P0DTD1 did not present results
Group 3 of P0DTD1 did not present results


In [12]:
fragsys_results_df.shape # total number of protein segments

(37, 13)

In [13]:
fragsys_results_df.head(3)

,acc,group,n_strucs,n_ligs,n_un_ligs,n_bs,n_seqs,n_human_seqs,n_var_seqs,n_vars,n_human_res,vars_per_seq,vars_per_res
0,P15379,0,24,25,24,5,108,24,20,1159,2098,48.29,0.55
1,Q96HY7,0,5,10,5,5,185,7,7,2429,4067,347.00,0.60
2,Q9Y2J2,0,51,56,51,10,214,56,54,5860,12756,104.64,0.46


In [9]:
fragsys_results_df.to_pickle(os.path.join(results_dir, "all_prots.pkl")) # all protein segments in data set

## BINDING SITES DATAFRAME

In [14]:
bs_dfs = []
for prot_dir in prot_dirs:
    prot = str(prot_dir).split("/")[-1]
    subdirs = sorted(os.listdir(os.path.join(prot_dir, "unsupp_cifs")))
    for subdir in subdirs:
        bs_res_path = os.path.join(prot_dir, "results", "{}_{}_BS_df.csv".format(prot, subdir))
        if os.path.isfile(bs_res_path):
            prot_bs_res = pd.read_csv(bs_res_path)
            prot_bs_def = pd.read_csv(os.path.join(prot_dir, "results", "{}_{}_BS_def_OC_single_i_rel_0.66.csv".format(prot, subdir)))
            bs_lig_occ_dict = {"BS"+str(k):v for k, v in prot_bs_def.binding_site.value_counts().to_dict().items()}
            prot_bs_res["number_ligs"] = prot_bs_res.bs_id.map(bs_lig_occ_dict)
            prot_bs_res["prop_ligs"] = prot_bs_res.number_ligs/prot_bs_res.number_ligs.sum()
            prot_bs_res["protein"] = prot
            prot_bs_res["group"] = subdir
            bs_dfs.append(prot_bs_res)
        else:
            print("Group {} of {} did not present binding sites data".format(prot, subdir))
            pass
all_bss_dfs = round(pd.concat(bs_dfs).reset_index(drop = True), 4)

Group P0DTD1 of 1 did not present binding sites data
Group P0DTD1 of 3 did not present binding sites data


In [15]:
all_bss_dfs.shape # total number of residues with structural information

(293, 14)

In [16]:
all_bss_dfs.head(3)

,bs_id,vars,occ,vars_per_occ,MES,p,norm_shenkin_rel,shenkin_ci,MES_ci,number_bs_res,number_ligs,prop_ligs,protein,group
0,BS0,23,37,0.6216,0.1203,0.6838,14.7446,14.3426,0.5255,8,2,0.08,P15379,0
1,BS1,6,14,0.4286,-0.2554,0.8153,7.9722,3.9064,0.9591,5,3,0.12,P15379,0
2,BS2,108,176,0.6136,0.1153,0.3647,41.1386,14.1057,0.2511,17,18,0.72,P15379,0


In [21]:
all_bss_dfs.to_pickle(os.path.join(results_dir, "all_bss.pkl")) # all binding sites

## BINDING RESIDUES DATAFRAME

In [17]:
fragsys_dfs = []
for prot_dir in prot_dirs:
    prot = str(prot_dir).split("/")[-1]
    subdirs = sorted(os.listdir(os.path.join(prot_dir, "unsupp_cifs")))
    for subdir in subdirs:
        res_path = os.path.join(prot_dir, "results", "{}_{}_fragsys_df.csv".format(prot, subdir))
        if os.path.isfile(res_path):
            prot_res = pd.read_csv(res_path)
            prot_res["protein"] = prot
            prot_res["group"] = subdir
            fragsys_dfs.append(prot_res)
        else:
            print("Group {} of {} did not present results".format(prot, subdir))
            pass
        
fragsyss_df = pd.concat(fragsys_dfs).reset_index(drop = True)
fragsyss_df.SS = fragsyss_df.SS.fillna("C")

Group P0DTD1 of 1 did not present results
Group P0DTD1 of 3 did not present results


In [18]:
fragsyss_df.shape # total number of ligand binding residues

(14172, 62)

In [19]:
fragsyss_df.head(3)

,index,UniProt_ResNum,Pfam_column,alignment_column,shenkin,occ,gaps,occ_pct,gaps_pct,rel_norm_shenkin,...,BS14,BS15,BS16,BS17,BS18,BS19,BS20,BS21,BS22,BS23
0,8,31,11,11,51.846631,57,51,0.527778,0.472222,82.865612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13,36,18,18,14.127808,97,11,0.898148,0.101852,14.690627,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,35,58,44,44,58.025383,107,1,0.990741,0.009259,94.033413,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
fragsyss_df.to_pickle(os.path.join(results_dir, "all_bs_ress.pkl")) # all binding site residues

## DSSP DATAFRAME

In [20]:
dssp_dfs = []
for prot_dir in prot_dirs:
    prot = str(prot_dir).split("/")[-1]
    subdirs = sorted(os.listdir(os.path.join(prot_dir, "unsupp_cifs")))
    for subdir in subdirs:
        dssp_subdir = os.path.join(prot_dir, "results", "dssp", subdir)
        sifts_subdir = os.path.join(prot_dir, "results", "sifts", subdir)
        dssp_files = [file for file in os.listdir(dssp_subdir) if file.startswith("dssp")]
        pdb_id = dssp_files[0].split("_")[1]
        example_dssp = os.path.join(dssp_subdir, dssp_files[0]) #only grabbing one example dssp file
        example_sifts = os.path.join(sifts_subdir, "sifts_mapping_{}.csv".format(pdb_id))
        if os.path.isfile(example_dssp):
            dssp_df = pd.read_csv(example_dssp)
            sifts_df = pd.read_csv(example_sifts)
            dssp_df["protein"] = prot
            dssp_df["group"] = subdir
            dssp_df["structure"] = pdb_id
            merged_df = dssp_df.merge(sifts_df[["PDB_ResNum", "PDB_ChainID", "UniProt_ResNum"]], how = "left", left_on = ["PDB_ResNum", "CHAIN"], right_on = ["PDB_ResNum", "PDB_ChainID"])
            dssp_dfs.append(merged_df)
        else:
            print("Group {} of {} did not present DSSP data".format(prot, subdir))
            pass
        
all_dssp_dfs = pd.concat(dssp_dfs)
all_dssp_dfs_filt = all_dssp_dfs.copy().query('UniProt_ResNum == UniProt_ResNum')
all_dssp_dfs_filt.UniProt_ResNum = all_dssp_dfs_filt.UniProt_ResNum.astype(int)

In [21]:
all_dssp_dfs_filt.shape # total number of residues with structural information

(13016, 17)

In [22]:
all_dssp_dfs_filt.head(3)

,PDB_ResNum,CHAIN,AA,SS,ACC,TCO,KAPPA,ALPHA,PHI,PSI,CHAIN_FULL,RSA,protein,group,structure,PDB_ChainID,UniProt_ResNum
1,25,A,Q,E,82,-0.950,360.0,-166.5,-135.2,146.3,A,41.414,P15379,0,5sc6,A,23
2,26,A,I,E,4,-0.999,10.9,-155.9,-129.4,128.5,A,2.367,P15379,0,5sc6,A,24
3,27,A,D,E,37,-0.900,6.8,-167.5,-101.2,132.1,A,22.699,P15379,0,5sc6,A,25


In [17]:
all_dssp_dfs_filt.to_pickle(os.path.join(results_dir, "all_dssp_dfs.pkl")) # all dssp data frames

## VARIANTS DATAFRAME

In [23]:
missense_dfs = []
for prot_dir in prot_dirs:
    prot = str(prot_dir).split("/")[-1]
    subdirs = sorted(os.listdir(os.path.join(prot_dir, "unsupp_cifs")))
    for subdir in subdirs:
        missense_df_path = os.path.join(prot_dir,"results/varalign/{}/{}_{}_missense_df.csv".format(subdir, prot, subdir))
        if os.path.isfile(missense_df_path):
            miss_df = pd.read_csv(missense_df_path)
            miss_df["protein"] = prot
            miss_df["group"] = subdir
            missense_dfs.append(miss_df)
        else:
            print("Group {} of {} did not present variation data".format(prot, subdir))
            pass
        
all_missense_df = pd.concat(missense_dfs).reset_index(drop = True)

Group P0DTD1 of 1 did not present variation data
Group P0DTD1 of 3 did not present variation data


In [24]:
all_missense_df.shape # total number of alignment columns with variation data

(10408, 17)

In [25]:
all_missense_df.head(3)

,col,shenkin,occ,gaps,occ_pct,gaps_pct,variants,rel_norm_shenkin,abs_norm_shenkin,oddsratio,log_oddsratio,pvalue,ci_dist,miss_class,miss_color,protein,group
0,1,6.000000,1,20,0.00,1.00,1,0.000000,0.000000,1.810181,0.593427,1.000000,2.772787,CME,green,P15379,0
1,2,9.639785,3,18,0.10,0.90,2,6.578738,3.192794,1.206678,0.187871,1.000000,1.790666,CME,green,P15379,0
2,3,11.927558,4,17,0.15,0.85,3,10.713779,5.199612,1.358038,0.306041,0.704905,1.498695,CME,green,P15379,0


In [29]:
all_missense_df.to_pickle(os.path.join(results_dir, "all_miss_dfs.pkl")) # all variant-containing columns

## INTERACTIONS DATAFRAME

In [26]:
all_cons_dfs = []
for prot_dir in prot_dirs:
    prot = str(prot_dir).split("/")[-1]
    subdirs = sorted(os.listdir(os.path.join(prot_dir, "unsupp_cifs")))
    for subdir in subdirs:
        arpeggio_subdir = os.path.join(prot_dir, "results", "arpeggio", subdir)
        arpeggio_files = [file for file in os.listdir(arpeggio_subdir) if file.startswith("arpeggio_all_cons_split")]
        for file in arpeggio_files:
            pdb_id = file[-8:-4]
            file_df = pd.read_csv(os.path.join(arpeggio_subdir, file))
            file_df["struc"] = pdb_id
            file_df["protein"] = prot
            all_cons_dfs.append(file_df)
all_fragsys_cons = pd.concat(all_cons_dfs).reset_index(drop = True)

In [27]:
all_fragsys_cons.shape # total number of interactions

(269013, 30)

In [28]:
all_fragsys_cons.head(3)

,Chain (Atom1),ResNum (Atom1),ResName (Atom1),Atom (Atom1),Chain (Atom2),ResNum (Atom2),ResName (Atom2),Atom (Atom2),Clash,Covalent,...,Carbonyl,Polar,Weak Polar,Atom proximity,Vdw proximity,Interacting entities,contact_type,UniProt_Resnum,struc,protein
0,A,33,ARG,CD,B,201,NW4,O2,0,0,...,0,0,0,3.771,0.551,INTER,sidechain,31.0,5sbv,P15379
1,A,33,ARG,CZ,B,201,NW4,O2,0,0,...,0,0,0,4.749,1.529,INTER,sidechain,31.0,5sbv,P15379
2,A,33,ARG,NE,B,201,NW4,O2,0,0,...,0,0,0,3.821,0.751,INTER,sidechain,31.0,5sbv,P15379


In [25]:
all_fragsys_cons.to_pickle(os.path.join(results_dir, "all_bss_cons.pkl")) # all binding site contacts